In [ ]:
import torch
import torch.nn as nn
import tiktoken
from model import GPTModel

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, #słownik
    "context_length": 256, #kontekst
    "emb_dim": 768, #osadzenia
    "n_heads": 12, #głowice uwagi
    "n_layers": 12, #warstwy
    "drop_rate": 0.1, #porzucenie
    "qkv_bias": False
}

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()